In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import sklearn
import _pickle as cPickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import learning_curve

In [14]:
source = open('data.csv', 'r')
reader = csv.reader(source)
next(reader)
i = 0
subjects = []
contents = []
labels = []

for line in reader:
    i += 1
    text = line[0]
    label = line[1]
    try:
        subject = text[text.index("Subject") + len("Subject:")
                    : text.index("  ")].strip()
        content = text[text.index("  ") :].strip()
    except ValueError:
        subject = text[text.index("Subject") + len("Subject:")
                    :]
        content = ""
    subjects += [subject]
    contents += [content]
    labels.append(label)

df = pd.DataFrame({'subject': subjects,
                   'content': contents,
                   'label': labels})
df['label'] = pd.to_numeric(df['label'])

In [12]:
def split_ttv(data, test_ratio, val_ratio):
    shuffled_indices = np.random.permutation(len(data),)
    test_set_size = int(len(data) * test_ratio)
    val_set_size = int(len(data) * val_ratio)
    test_indices = shuffled_indices[:test_set_size]
    val_indices = shuffled_indices[test_set_size:
        test_set_size + val_set_size]
    train_indices = shuffled_indices[test_set_size + val_set_size:]
    return (data.iloc[train_indices],
            data.iloc[test_indices],
            data.iloc[val_indices])

In [9]:
train, test, validation = split_ttv(df, 0.2, 0.1)
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)
validation.to_csv('validation.csv', index=False)